In [1]:
# Importing Modules

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
from bs4 import BeautifulSoup
import os
import pdfkit
from time import time
import threading

In [2]:
# Getting All site Information
req = urllib3.PoolManager()
url = "https://www.javatpoint.com"
res = req.request('GET', url)
soup = BeautifulSoup(res.data, 'html.parser')
allContent = soup.findAll('fieldset', attrs={'class': 'gra1'})

In [3]:
# Manipulating Site Information 

fieldNames = []
topics = []
for field in allContent:
    headline = field.find('h2').text
    fieldNames.append(headline)
    contentNames = field.findAll('p')
    contentLinks = field.findAll('a')
    fields = []
    for i in range(len(contentNames)):
        if(contentLinks[i]['href'][0:4]!='http'):
            contentLinks[i]['href'] = 'http://www.javatpoint.com/' + contentLinks[i]['href']
        fields.append((contentNames[i].text, contentLinks[i]['href']))
    topics.append(fields)

In [4]:
# Define User choice

def getIndex(fieldNames):
    try:
        ind = int(input('Select one index: '))
        if(ind>0 and ind<=len(fieldNames)):
            return ind
        else:
            print('Please be a human!')
            return getIndex(fieldNames)
    except:
        print('Please Select a number')
        return getIndex(fieldNames)

In [5]:
# All Topics

for i in range(len(fieldNames)):
    print('{0}. {1}'.format(i+1, fieldNames[i]))
    for x in topics[i]:
        print('\t{0}'.format(x[0]))

1. Latest Tutorials
	Agile
	QA
	Angular 7
	D. Science
	ReactJS
	Software E.
2. Trending Technologies
	AI
	AWS
	Selenium
	IoT
	Cloud
	Hadoop
3. Preparation
	Aptitude
	Reasoning
	Verbal A.
	Interview
	Company
4. B.Tech / MCA
	DBMS
	DS
	DAA
	OS
	C. Network
	Compiler D.
	COA
	D. Math.
	E. Hacking
	C. Graphics
	Web Tech.
	Cyber Sec.
	Automata
	C
	C++
	Java
	.Net
	Python
	Programs
	Control S.
5. Popular Tutorials
	DS
	Java
	C#
	C
	C++
	SQL
	Android
	Python
	Excel
	Computer
	Linux
	vi editor
	Embedded
	Robotics
	SEO
	F#
	Powerpoint
	Shell Script
	MathML
	Perl
	Scala
	MS Word
	Ruby
	Rails
	Docker
	ASP.NET
	ADO.NET
	GWT
	Go Lang
	On Ubuntu
	Phalcon
	On Mac
	On CentOS
	SVG
	Polymer
	Compiler D.
	Django
	.Net
	Kotlin
	Aptitude
	Rust
	Control S.
	Tkinter
	TypeScript
	React Native
	NumPy
	Testing
	Flask
	Pig
	Swift
	Spark
	TestNG
	DIP
	Mobile Com.
	SAS
	Electrical M.
6. Database Tutorials
	SQL
	PL/SQL
	Oracle
	MySQL
	PostgreSQL
	MongoDB
	Cassandra
	SQLite
	Neo4j
	CouchDB
	SQL Server
	MariaDB
	DB2
	

In [6]:
# Get User choice

ind1 = getIndex(fieldNames)
for i in range(len(topics[ind1-1])):
    print('{0}. {1}'.format(i+1, topics[ind1-1][i][0]))
ind2 = getIndex(topics[ind1-1])

topicNameDecided = topics[ind1-1][ind2-1][0]
topicLinkDecided = topics[ind1-1][ind2-1][1]
print(topics[ind1-1][ind2-1][0], topics[ind1-1][ind2-1][1])

Select one index: 5
1. DS
2. Java
3. C#
4. C
5. C++
6. SQL
7. Android
8. Python
9. Excel
10. Computer
11. Linux
12. vi editor
13. Embedded
14. Robotics
15. SEO
16. F#
17. Powerpoint
18. Shell Script
19. MathML
20. Perl
21. Scala
22. MS Word
23. Ruby
24. Rails
25. Docker
26. ASP.NET
27. ADO.NET
28. GWT
29. Go Lang
30. On Ubuntu
31. Phalcon
32. On Mac
33. On CentOS
34. SVG
35. Polymer
36. Compiler D.
37. Django
38. .Net
39. Kotlin
40. Aptitude
41. Rust
42. Control S.
43. Tkinter
44. TypeScript
45. React Native
46. NumPy
47. Testing
48. Flask
49. Pig
50. Swift
51. Spark
52. TestNG
53. DIP
54. Mobile Com.
55. SAS
56. Electrical M.
Select one index: 18
Shell Script http://www.javatpoint.com/shell-scripting-tutorial


In [7]:
# Getting Modules

url = topicLinkDecided
res = req.request('GET', url)
soup = BeautifulSoup(res.data, 'html.parser')

In [8]:
# Creating Modules

moduleTags = soup.findAll('h2', attrs={'class': 'spanh2'})
modules = []
for module in moduleTags:
    modules.append(topicNameDecided+'/'+module.text)

for module in modules:
    if not os.path.exists(module):
        os.makedirs(module)

In [9]:
def createHTMLFromLinks(links,i):
    req = urllib3.PoolManager()
    global reqNo
    for link in links:
        reqNo+=1
        print("Request Number:" + str(reqNo))
        url = "http://www.javatpoint.com/" + link
        r = req.request('GET', url)
        soup = BeautifulSoup(r.data, 'html.parser')
        [s.extract() for s in soup('script')]  # Remove Scripts
        [s.extract() for s in soup('ins')]     # Remove Ads
        [s.extract() for s in soup('iframe')]  # Remove Ads2
        mainContent = soup.find('div', attrs={'id': 'city'})

        # For Images
        images = mainContent.findAll('img')
        for x in range(len(images)):
            imageSrc = images[x]['src']
            imageSrc = str(imageSrc)
            if(imageSrc.find('javatpoint.com')==-1):
                images[x]['src'] = 'http://javatpoint.com/' + images[x]['src']

        for x in range(len(images)):
            mainContent.findAll('img')[x]['src']=images[x]['src']
            
        # For CodeBlock
        codes = mainContent.findAll('textarea', attrs= {'name': 'code'})
        for c in range(len(codes)):
            newTag = soup.new_tag('p')
            newTag.append(str(codes[c].get_text()))
            mainContent.find('textarea').replaceWith(newTag)

        # Final HTML
        mainContent = str(mainContent).replace("\n"," ")
        header = "<html> <head> <meta charset='UTF-8'> <link rel=\"stylesheet\" type=\"text/css\" href=\"./../../javatpoint.css\"> </head>  <body>"
        footer = "</body></html>"
        mainContent = header + mainContent + footer
        try:
            f = open("./" + modules[i] + "/" + link + ".html", "w")
            for ch in mainContent:
                try:
                    f.write(ch)
                except:
                    f.write(" ")
            f.close()
        except:
            print("Can't Create " + str(link) + ".html")

In [12]:
# Creating HTML files from each modules
reqNo=0
contentTable = soup.findAll('div', attrs={'class': 'leftmenu'}) # Index

start = time()
threads = []
for i in range(len(contentTable)):
    links = []
    for x in contentTable[i]:
        a = str(x)
        y = "href="
        i1 = a.find(y)
        if(i1==-1): # No Links
            continue
        i1+=6 # Skip <"a href=">
        i2 = a.find('\"', i1)  # Find Ending quote
        links.append(a[i1:i2]) # a[i1:i2] is the final href
    print(links)
    t1 = threading.Thread(target=createHTMLFromLinks, args=(links,i,))
    threads.append(t1)

print("Threads are executing")

for i in range(len(threads)):
    threads[i].start()

for i in range(len(threads)):
    threads[i].join()
    
print("Threads finished executing")

end = time()
print("Time Taken: "+ str(end - start))

['shell-scripting-tutorial', 'what-is-shell-scripting', 'how-to-determine-shell-scripting', 'shell-scripting-she-bang', 'shell-scripting-comments', 'shell-scripting-variables', 'shell-scripting-sourcing-a-file', 'troubleshooting-a-shell-script', 'shell-scripting-prevent-setuid-root-spoofing']
['steps-to-write-and-execute-a-shell-script']
['shell-script-parameters', 'shell-scripting-shift-through-parameters']
['shell-scripting-sourcing-a-config-file']
['get-shell-script-options-with-getopts']
['if-then-else-shell-scripting', 'if-then-elif-shell-scripting', 'for-loop-shell-scripting', 'while-loop-shell-scripting', 'until-loop-shell-scripting']
['scripting-shell-function', 'shell-scripting-case', 'shell-scripting-eval', 'shell-let-scripting']
Threads are executing
Request Number:1
Request Number:2
Request Number:3
Request Number:4
Request Number:5Request Number:6

Request Number:7
Request Number:8
Request Number:9
Request Number:10
Request Number:11
Request Number:12
Request Number:13
Req

In [13]:
# Sorting HTML according to creation time
def sorted_dir(folder):
    def getctime(name):
        path = os.path.join(folder, name)
        return os.path.getctime(path)
    return sorted(os.listdir(folder), key=getctime)

# For Windows only
# path_wkthmltopdf = "C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe"
# config = pdfkit.configuration(wkhtmltopdf=path_wkthmltopdf)
def convertToPdf(myList,module_name):
    pdfkit.from_file(myList, module_name + '.pdf')

In [15]:
# Converting each module to PDF
start = time()
threads = []
for x in range(len(modules)):
    myList = sorted_dir(modules[x])
    for i in range(len(myList)):
        myList[i]=modules[x] + "/" + myList[i]
    t1 = threading.Thread(target=convertToPdf, args=(myList,modules[x],))
    threads.append(t1)

end = time()
print("Threads are executing")

for i in range(len(threads)):
    threads[i].start()
    
for i in range(len(threads)):
    threads[i].join()
    
print("Threads finished executing")

end = time()
print("Time Taken: "+ str(end - start))

Threads are executing
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                   